## 1. Importação dos dados de Licitações

Primeiro importarmos a planilha, estamos focando nas licitações recentes, portanto 2016 e 2017.

In [322]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sys
import math
import re
import regex
import nltk 
from  nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

%matplotlib inline
pal = sns.color_palette()
corpus = []
word_set = set()

lics = pd.read_csv('./input/licitacoes_2016_2017.csv', dtype={'Fornecedor Documento': object})
#del lics['Unnamed: 15']
#lics.to_csv('./input/licitacoes_2016_2017.csv', index=False, encoding='utf-8')
lics.head(2)

,Orgão,Retranca,Modalidade,Número Licitação,Número Processo,Evento,objeto,Data Publicação Extrato,Fornecedor,Fornecedor Tipo,Fornecedor Documento,Data Assinatura Extrato,Validade Extrato,Tipo Validade Extrato,Número Contrato,Valor Contrato
0,EDUCAÇÃO,EGAAADM,PREGÃO ELETRÔNICO,14/SME/2015,2014-0.286.506-2,EXTRATO DE CONTRATO / NOTA DE EMPENHO,REGISTRO DE PREÇOS PARA AQUISIÇÃO DE FÓRMULA I...,04/03/2016,MARTINUCI COMÉRCIO E REPRESENTAÇÕES DE PRODUTO...,PJ,18097272000117,24/02/2016,4.0,Meses,10/SME/CODAE/2016,"942,955.20"
1,HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL,ELAAADM,PREGÃO ELETRÔNICO,145/2015,2015-0.223.234-7,EXTRATO DE CONTRATO / NOTA DE EMPENHO,FORMULAÇÕES MAGISTRAIS E OFICINAIS,04/03/2016,PIRES DE CAMPOS & CIA LTDA - EPP,PJ,45516507000130,29/02/2016,12.0,Meses,078/2016,"55,944.00"


## 2. Formatação dos dados

Aqui nós formatamos os dados que serão usados nos cálculos referente as suspeitas. Os campos formatados são:
- **Data Assinatura**: convertido para milisegundos.
- **Validade**: convertido para dias. Consideramos em média que 1 mês = 30,417 dias, que seria a média em 1 ano de 365 dias.
- **Valor Contrato**: convertido para float.
- **ID Contrato**: Definimos um ID para cada contrato.
- **Evento**: Simplificamos as opções de eventos.
    

In [327]:
from datetime import datetime
import time

def unix_time(dt):
    epoch = datetime.fromtimestamp(0)
    delta = dt - epoch
    return delta.total_seconds()

def unix_time_millis(dt):
    return int(unix_time(dt) * 1000)

#lics['data_assinatura'] = 0
#lics['validade'] = 0
#del lics['contrato_id']
#lics['contrato_id'] = None
for index, row in lics.iterrows():
    if row['Evento']:
        evento = row['Evento'].replace('EXTRATO DE CONTRATO','')
        evento = evento.replace(' / ','')
        evento = evento.replace('EXTRATO DE ATA DE ','')
        evento = evento.replace('EXTRATO DE ','')
        evento = evento.replace('EXTRATO DO ','')
        lics.set_value(index,'Evento',evento.strip())
    lics.set_value(index,'contrato_id',str(index))
    if row['Data Assinatura Extrato']:
        datetime_object = datetime.strptime(row['Data Assinatura Extrato'], '%d/%m/%Y')
        data = unix_time_millis(datetime_object)
        lics.set_value(index,'data_assinatura',data)
    if row['Validade Extrato']:
        if isinstance(row['Tipo Validade Extrato'], str):
            if isinstance(row['Validade Extrato'], float):
                dias = row['Validade Extrato']
                if row['Tipo Validade Extrato'].lower() == 'meses':
                    dias = dias * 30.417
                if row['Tipo Validade Extrato'].lower() == 'anos':
                    dias = dias * 365
                lics.set_value(index,'validade',round(dias,2))
            else:
                lics.set_value(index,'validade',0)
        else:
            lics.set_value(index,'validade',0)
    if row['Valor Contrato']:
        if isinstance(row['Valor Contrato'], str):
            valor = row['Valor Contrato'].replace(",", "")
            lics.set_value(index,'valor',round(float(valor), 2))
        else:
            lics.set_value(index,'valor',0)

lics['data_assinatura'].astype(int).fillna(0)
lics['validade'].astype(float).fillna(0.0)
lics['valor'].astype(float).fillna(0.0)
lics.to_csv('./output/licitacoes_2016_2017.csv', index=False, encoding='utf-8')
lics.head(2)
#orgaos = lics['Orgao'].tolist()
#print(sorted(set(orgaos)))

,Orgão,Retranca,Modalidade,Número Licitação,Número Processo,Evento,objeto,Data Publicação Extrato,Fornecedor,Fornecedor Tipo,Fornecedor Documento,Data Assinatura Extrato,Validade Extrato,Tipo Validade Extrato,Número Contrato,Valor Contrato,data_assinatura,validade,valor,contrato_id
0,EDUCAÇÃO,EGAAADM,PREGÃO ELETRÔNICO,14/SME/2015,2014-0.286.506-2,NOTA DE EMPENHO,REGISTRO DE PREÇOS PARA AQUISIÇÃO DE FÓRMULA I...,04/03/2016,MARTINUCI COMÉRCIO E REPRESENTAÇÕES DE PRODUTO...,PJ,18097272000117,24/02/2016,4.0,Meses,10/SME/CODAE/2016,"942,955.20",1.456283e+12,121.67,942955.2,0
1,HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL,ELAAADM,PREGÃO ELETRÔNICO,145/2015,2015-0.223.234-7,NOTA DE EMPENHO,FORMULAÇÕES MAGISTRAIS E OFICINAIS,04/03/2016,PIRES DE CAMPOS & CIA LTDA - EPP,PJ,45516507000130,29/02/2016,12.0,Meses,078/2016,"55,944.00",1.456715e+12,365.00,55944.0,1


## 3. Método de Pré-Processamento

O objetivo deste método é preparar o "Objeto" das licitações para ser treinado em um modelo de rede neural, que usaremos a seguir. No momento, temos 3 atividades de Pré-Processamento dos dados:
- Removemos caracteres especiais
- Convertemos todas palavras em minúsculas e as separamos em um array.
- Removemos todas palavras que são "stop words" para não influenciar o modelo a ser treinado. Exemplo de "stop words": de, para, a, no, por, num, minha.

In [3]:
def preprocessing(raw):
    if str(raw) == 'nan':
        return ""
    try:
        # 1. Remove non-letters
        raw = regex.sub("[^\p{Latin}]"," ", raw, re.UNICODE)

        # 2. Convert words to lower case and split them
        raw = [(re.sub(r'((?<!\d)\.(?!\d))|((?<!\d)\:(?!\d))', '', word)).replace(',','') for word in raw.lower().split() if len(word) > 1]
        #raw = raw.lower().split()
        #raw = re.findall(r'\w+|[.]+', raw.lower())

        # 3. Remove stop words (false by default)
        stops = set(stopwords.words("portuguese"))
        #wordlist = nltk.word_tokenize(raw)        
        raw = [w for w in raw if not w in stops]
    except:
        print("Unexpected error:", sys.exc_info())
        
    return raw

## 4. Método de Treinamento do Modelo de Rede Neural

Definimos aqui o método que irá realizar o treinamento do modelo usado para identificar similaridades entre "Objeto" de Licitações. Estamos usando o algoritmo Doc2Vec da biblioteca gensim, pois tem demonstrado melhores resultados comparado a outros algoritmos como "bag of words" e TF-IDF.

In [4]:
# Doc2Vec
def custom_Doc2Vec(taggeddocs):
    # Building the model
    model = Doc2Vec(taggeddocs, dm=0, alpha=0.025, size=20, min_alpha=0.025, min_count=0)
    start = time.perf_counter()
    print('Started Training')

    # Training
    total_epochs = 80
    total_docs = len(taggeddocs)
    for epoch in range(total_epochs):
        started_epoch = time.time()
        t0 = time.perf_counter()
        if epoch % 10 == 0:
            print("Started Epoch %d" % (epoch + 1))
        model.train(taggeddocs, total_examples=total_epochs, epochs=total_epochs)
        model.alpha -= 0.002 # decrease the learning rate
        model.min_alpha = model.alpha # fix the learning rate, no decay
        if epoch % 10 == 0:
            t1 = time.perf_counter()
            print("Epoch %d in: " % (epoch + 1), "%.3f seconds" % (t1-t0))
            
    end = time.perf_counter()
    print('Finished Training in: ', "%.3f seconds" % (end-start))
    
    return model

## 5. Execução do Treinamento do Modelo

Com o "Objeto" pré-processado e convertido para um array de palavras, agora nós executamos o modelo. E em seguida executamos um teste para identificar a similaridade entre os "Objetos" de 2 licitações quaisquer. E finalmente salvamos o modelo treinado para usarmos depois.

In [309]:
taggeddocs = []
for index, lic in lics.iterrows():
    licx = preprocessing(lic['objeto'])
    if len(licx) > 0:
        doc = TaggedDocument(words=licx, tags=[lic['contrato_id']])
        taggeddocs.append(doc)

model = custom_Doc2Vec(taggeddocs)
#model = Doc2Vec.load('./output/lics_model.doc2vec')

In [311]:
id1 = 2947
id2 = 10580
text1 = preprocessing(lics['objeto'].tolist()[id1])
text2 = preprocessing(lics['objeto'].tolist()[id2])
# Comput Cosine distances
similarity_vec = model.n_similarity(text1, text2)
print('obj 1 => ', lics.loc[id1]['objeto'])
print('\nobj 2 => ', lics.loc[id2]['objeto'])
print('\nSimilarity Index: {:4.2f} %'.format(similarity_vec*100))

obj 1 =>  PRESTAÇÃO DE SERVIÇOS DE NUTRIÇÃO E ALIMENTAÇÃO HOSPITALAR, VISANDO O FORNECIMENTO DE DIETAS GERAIS, DIETAS ESPECIAIS, DIETAS ENTERAIS E FÓRMULAS LÁCTEAS DESTINADAS A PACIENTES (ADULTOS E INFANTIS); ACOMPANHANTES LEGALMENTE INSTITUÍDOS; PACIENTES PERTENCENTES AO PROHDOM (PROGRAMA DE HOSPITALIZAÇÃO DOMICILIAR) ,  RESIDENTES E DEMAIS USUÁRIOS DEVIDAMENTE AUTORIZADOS; ASSEGURANDO UMA ALIMENTAÇÃO BALANCEADA E EM CONDIÇÕES HIGIÊNICO-SANITÁRIAS ADEQUADAS, ENGLOBANDO A OPERACIONALIZAÇÃO E DESENVOLVIMENTO DE TODAS AS ATIVIDADES DE PRODUÇÃO, TRANSPORTE, DISTRIBUIÇÃO E ADMINISTRATIVAS,INCLUINDO NUTRIÇÃO CLÍNICA.

obj 2 =>  PRESTAÇÃO DE SERVIÇOS DE NUTRIÇÃO E ALIMENTAÇÃO HOSPITALAR, VISANDO O FORNECIMENTO DE DIETAS GERAIS, DIETAS ESPECIAIS, DIETAS ENTERAIS (FORNECIMENTO, ENVASE E DISTRIBUIÇÃO) E FÓRMULAS LÁCTEAS DESTINADAS A PACIENTES (ADULTOS E INFANTIS); ACOMPANHANTES LEGALMENTE INSTITUÍDOS (LEI FEDERAL No 8.069 DE 13/07/90; ART. 278, INC. VII DA CONSTITUIÇÃO DO ESTADO DE SÃO PAULO; L

In [ ]:
model.save('./output/lics_model.doc2vec')

## 6. Identificação de Similaridade e Suspeitas

Com o modelo treinado, podemos agora identificar a similaridade do "Objeto" entre todas as Licitações. Primeiro, caso 2 ou mais licitações tenha no minímo 90% de probabilidade de serem similares, vamos relacioná-las na coluna "similar" da planilha original das licitações. Segundo, vamos identificar quais licitações são consideras suspeitas, e para isso tivemos que identificar alguns dados primeiro:
- **Valor/Dia**: Valor Total do contrato dividido pela quantidade de dias referente a prestação do serviço ou aquisição.
- **Taxa**: Valor/Dia de um determinado contrato dividido pelo Valor/Dia de outro contrato considerado similar.
- **Intervalo Dias**: diferença em dias entre o início de um contrato e de outro contrato considerado similar.
- **Correção**: intervalo de dias identificado acima vezes uma taxa de juros. Neste momento estamos considerando uma taxa de 14,6% ao ano, que seria em média 0,04% ao dia.
- **Valor/Dia Corrigido**: Valor/Dia vezes a Correção identificada acima.
- **Taxa Corrigida**: Valor/Dia corrigido, identificada acima, dividido pelo Valor/Dia do contrato similar.
- **Diferença de Valor**: Valor/Dia corrigido, identificada acima, menos o Valor/Dia do contrato similar, e depois multiplicado pela quantidade de dias.

E com os dados acima, consideramos os seguintes critérios para definir licitações como suspeitas:
- O "Objeto" de 2 licitações tenha no minímo 95% de probabilidade de similaridade.
- A Taxa Corrigida identifique que o valor do contrato corrigido está 20% ou mais acima do contrato similar.

Ao final, salvamos a planilha atualizada.

In [334]:
t0 = time.perf_counter()
licitacoes = {}
suspeitas = {}
for index, lic1 in lics.iterrows():
    lic1x = preprocessing(lic1['objeto'])
    if len(lic1x) > 0:
        i = 0
        valor_dia_x = 0
        valor_x = float(lic1['valor'])
        dias_x = lic1['validade']
        if dias_x > 0:
            valor_dia_x = round(valor_x / dias_x, 2)
        for sim in model.docvecs.most_similar([int(lic1['contrato_id'])], topn=50):
            if sim[1] >= 0.9:
                if i == 0:
                    #print('>>>> ', lic1['contrato_id'], valor_dia_x)
                    i = i + 1
                if str(lic1['contrato_id']) not in licitacoes:
                    licitacoes[str(lic1['contrato_id'])] = []
                valor_y = float(lics.loc[sim[0]]['valor'])
                dias_y = lics.loc[sim[0]]['validade']
                sim_y = {
                    'id': sim[0], 
                    'similar': round(sim[1],2)
                }
                if dias_y > 0 and valor_y > 0 and valor_dia_x > 0:
                    #print('> ', sim[0], valor_dia_y, ' => ', taxa_y, lics.loc[sim[0]]['Evento'])
                    valor_dia_y = float(str(round(valor_y/dias_y, 2)))
                    taxa_y = float(str(round(valor_dia_y / valor_dia_x, 2)))
                    intervalo_dias_y = (lics.loc[sim[0]]['data_assinatura'] - lic1['data_assinatura'])/86400000
                    correcao_y = 0.0004 * intervalo_dias_y
                    valor_dia_corrigido_y = valor_dia_y + (valor_dia_y * correcao_y)
                    taxa_corrigida_y = float(str(round(valor_dia_corrigido_y / valor_dia_x, 2)))
                    diferenca_y =  float(str(round( (abs(valor_dia_corrigido_y) - valor_dia_x) * dias_y, 2 )))
                    sim_y = {
                        'id': sim[0], 
                        'similar': round(sim[1],2),
                        'valor_dia': valor_dia_y,
                        'taxa': taxa_y,
                        'intervalo_dias': intervalo_dias_y,
                        'valor_dia_corrigido': valor_dia_corrigido_y,
                        'taxa_corrigida': taxa_corrigida_y,
                        'diferenca': diferenca_y,
                        'tipo': lics.loc[sim[0]]['Evento']
                    }
                    if sim[1] >= 0.95 and taxa_corrigida_y > 1.2 and taxa_corrigida_y < 5.0:
                        #print('> ', lic1['contrato_id'], sim[0], diferenca_y, ' => ', taxa_corrigida_y)
                        if str(sim[0]) not in suspeitas:
                            suspeitas[str(sim[0])] = {'id':sim[0],'dif':diferenca_y,'id_2':lic1['contrato_id']}
                        elif diferenca_y > suspeitas[str(sim[0])]['dif']:
                            suspeitas[str(sim[0])] = {'id':sim[0],'dif':diferenca_y,'id_2':lic1['contrato_id']}
                        
                licitacoes[str(lic1['contrato_id'])].append(sim_y)
    if int(lic1['contrato_id']) % 1000 == 0:
        t1 = time.perf_counter()
        print("Done %d in: " % (int(lic1['contrato_id']) + 1), "%.3f seconds" % (t1-t0))
        t0 = time.perf_counter()

Done 1 in:  0.069 seconds
Done 1001 in:  12.054 seconds
Done 2001 in:  11.390 seconds
Done 3001 in:  16.692 seconds
Done 4001 in:  16.931 seconds
Done 5001 in:  14.630 seconds
Done 6001 in:  12.471 seconds
Done 7001 in:  16.079 seconds
Done 8001 in:  16.905 seconds
Done 9001 in:  14.642 seconds
Done 10001 in:  13.386 seconds
Done 11001 in:  10.915 seconds


In [335]:
lics['similar'] = ''
lics['id_suspeito'] = -1
lics['valor_suspeito'] = 0.0
for index, row in lics.iterrows():
    if str(index) in licitacoes:
        lics.set_value(index,'similar',licitacoes[str(index)])
    if str(index) in suspeitas:
        lics.set_value(index,'id_suspeito',suspeitas[str(index)]['id_2'])
        lics.set_value(index,'valor_suspeito',suspeitas[str(index)]['dif'])

lics.to_csv('./output/licitacoes_2016_2017.csv', index=False, encoding='utf-8')


## 7. Processamento e Envio para a AWS

Aqui definimos o método que irá enviar os dados para o DynamoDB na AWS.

In [385]:
import boto
from csv import reader

MY_ACCESS_KEY_ID = 'XXX'
MY_SECRET_ACCESS_KEY = 'XXX'


def do_batch_write(items, table_name, dynamodb_table, dynamodb_conn):
    batch_list = dynamodb_conn.new_batch_write_list()
    batch_list.add_batch(dynamodb_table, puts=items)
    while True:
        response = dynamodb_conn.batch_write_item(batch_list)
        unprocessed = response.get('UnprocessedItems', None)
        if not unprocessed:
            break
        batch_list = dynamodb_conn.new_batch_write_list()
        unprocessed_list = unprocessed[table_name]
        items = []
        for u in unprocessed_list:
            item_attr = u['PutRequest']['Item']
            item = dynamodb_table.new_item(
                    attrs=item_attr
            )
            items.append(item)
        batch_list.add_batch(dynamodb_table, puts=items)


def import_csv_to_dynamodb(table_name, csv_file_name, colunm_names,     column_types):
    dynamodb_conn =     boto.connect_dynamodb(aws_access_key_id=MY_ACCESS_KEY_ID, aws_secret_access_key=MY_SECRET_ACCESS_KEY)
    dynamodb_table = dynamodb_conn.get_table(table_name)     
    BATCH_COUNT = 25 # 25 is the maximum batch size for Amazon DynamoDB

    items = []

    count = 0
    csv_file = open(csv_file_name, 'r')
    for cur_line in reader(csv_file):
        count += 1
        if count == 1:
            continue

        row = {}
        for colunm_number, colunm_name in enumerate(colunm_names):
            if table_name == 'contratos':
                if colunm_name == 'user_id':
                    row[colunm_name] = column_types[colunm_number]  ('12345')
                else:
                    if colunm_name == 'data_assinatura' or colunm_name == 'validade':
                        if cur_line[colunm_number] == '':
                            cur_line[colunm_number] = 0
                    #print(colunm_name, cur_line[colunm_number])
                    row[colunm_name] = column_types[colunm_number]    (cur_line[colunm_number])
            else:
                row[colunm_name] = column_types[colunm_number]    (cur_line[colunm_number])

        if table_name == 'contratos':
            if len(row['orgao']) == 0:
                del row['orgao']
            if len(row['retranca']) == 0:
                del row['retranca']
            if len(row['modalidade']) == 0:
                del row['modalidade']
            if len(row['numero_licitacao']) == 0:
                del row['numero_licitacao']
            if len(row['numero_processo']) == 0:
                del row['numero_processo']
            if len(row['evento']) == 0:
                del row['evento']
            if len(row['objeto']) == 0:
                del row['objeto']
            if len(row['data_publicacao']) == 0:
                del row['data_publicacao']
            if len(row['fornecedor']) == 0:
                del row['fornecedor']
            if len(row['fornecedor_tipo']) == 0:
                del row['fornecedor_tipo']
            if len(row['fornecedor_doc']) == 0:
                del row['fornecedor_doc']
            if len(row['data_assinatura_s']) == 0:
                del row['data_assinatura_s']
            if len(row['validade_s']) == 0:
                del row['validade_s']
            if len(row['validade_tipo']) == 0:
                del row['validade_tipo']
            if len(row['numero_contrato']) == 0:
                del row['numero_contrato']
            if len(row['valor_s']) == 0:
                del row['valor_s']

            if row['data_assinatura'] == 0:
                del row['data_assinatura']
            if row['validade'] == 0:
                del row['validade']
            if row['valor'] == 0:
                del row['valor']
            if len(row['similar']) == 0:
                del row['similar']
            if row['id_suspeito'] == -1:
                del row['id_suspeito']
            if row['valor_suspeito'] == 0:
                del row['valor_suspeito']
            if len(row['orgao_id']) == 0:
                del row['orgao_id']
        else:
            if row['susp_s'] == 0:
                del row['susp_s']
            if row['susp_v'] == 0.0:
                del row['susp_v']
                
        #print(count, row)
        item = dynamodb_table.new_item(attrs=row)
        items.append(item)
        if count % BATCH_COUNT == 0:
            #print ('batch write start ... ', 
            do_batch_write(items, table_name, dynamodb_table, dynamodb_conn)
            items = []
            if count % 1000 == 0:
                print ('batch done! (row number: ' + str(count) + ')')

    # flush remaining items, if any
    if len(items) > 0: 
        do_batch_write(items, table_name, dynamodb_table, dynamodb_conn)


    csv_file.close() 


### 7.1 Dados de Orgãos

Primeiro vamos tratar e enviar os dados referente aos orgãos.

In [397]:
orgaos = []
orgaos_x = {}
orgs = lics['Orgão'].tolist()
orgs = set(orgs)
for index, value in enumerate(orgs):
    itens = lics.loc[lics['Orgão'] == value]
    tam = len(itens)
    tam_aditamentos = 0
    if tam > 0:
        ultimo_lic = 0.0
        susp_s = 0
        susp_v = 0.0
        valor_total = 0.0
        for i, item in itens.iterrows():
            if item['Evento'] == 'ADITAMENTO':
                tam_aditamentos = tam_aditamentos + 1
            if item['valor'] > 0:
                valor_total +=item['valor']
            if item['valor_suspeito'] > 0:
                susp_s = susp_s + 1
                susp_v += item['valor_suspeito']
            if item['data_assinatura'] > ultimo_lic:
                ultimo_lic = item['data_assinatura']
        orgao = {
            'orgao_id': index,
            'nome': value,
            'lics': tam,
            'ultimo_lic': int(ultimo_lic),
            'valor_total': valor_total,
            'susp_s': susp_s,
            'susp_v': susp_v,
            'adits': tam_aditamentos,
            'user_id': '12345'
        }
        orgaos_x[value] = index
        orgaos.append(orgao)
variables = list(orgaos[0].keys())
orgaos_df = pd.DataFrame([[i[j] for j in variables] for i in orgaos], columns = variables)
orgaos_df.to_csv('./output/orgaos_2016_2017.csv', index=False, encoding='utf-8')

In [401]:
lics['orgao_id'] = None
#del lics['id_orgao']
suspeitometro = 0.0
for index, row in lics.iterrows():
    lics.set_value(index,'orgao_id',orgaos_x[row['Orgão']])
    
    if row['valor_suspeito'] > 0:
        suspeitometro += row['valor_suspeito']

lics.to_csv('./output/licitacoes_2016_2017.csv', index=False, encoding='utf-8')
print(suspeitometro)

3839109615.6399965


In [402]:
colunm_names = 'orgao_id nome lics ultimo_lic valor_total susp_s susp_v adits user_id'.split()
table_name = 'orgaos'
csv_file_name = './output/orgaos_2016_2017.csv'
column_types = [str, str, int, int, float, int, float, int, str]
import_csv_to_dynamodb(table_name, csv_file_name, colunm_names, column_types)


### 7.2. Dados de Fornecedores

Agora tratamos e enviamos os dados referente aos fornecedores.

In [396]:
fornecedores = []
docs = lics['Fornecedor Documento'].tolist()
docs = set(docs)
#print(len(set(docs)))
for index, value in enumerate(docs):
    itens = lics.loc[lics['Fornecedor Documento'] == value]
    tam = len(itens)
    tam_aditamentos = 0
    if tam > 0:
        ultimo_lic = 0.0
        nome = None
        susp_s = 0
        susp_v = 0.0
        valor_total = 0.0
        for i, item in itens.iterrows():
            if item['Evento'] == 'ADITAMENTO':
                tam_aditamentos = tam_aditamentos + 1
            if item['valor'] > 0:
                valor_total +=item['valor']
            if item['valor_suspeito'] > 0:
                susp_s = susp_s + 1
                susp_v += item['valor_suspeito']
            if item['data_assinatura'] > ultimo_lic:
                nome = item['Fornecedor']
                tipo = item['Fornecedor Tipo']
                ultimo_lic = item['data_assinatura']
        if nome is not None and isinstance(nome, str):
            if len(nome) < 2:
                print(index, value)
            fornecedor = {
                'doc': value,
                'doc_tipo': tipo,
                'nome': nome,
                'lics': tam,
                'ultimo_lic': int(ultimo_lic),
                'valor_total': valor_total,
                'susp_s': susp_s,
                'susp_v': susp_v,
                'adits': tam_aditamentos,
                'user_id': '12345'
            }
            fornecedores.append(fornecedor)
variables = list(fornecedores[0].keys())
fornecedores_df = pd.DataFrame([[i[j] for j in variables] for i in fornecedores], columns = variables)
fornecedores_df.to_csv('./output/fornecedores_2016_2017.csv', index=False, encoding='utf-8')

In [408]:
colunm_names = 'doc doc_tipo nome lics ultimo_lic valor_total susp_s susp_v adits user_id'.split()
table_name = 'fornecedores'
csv_file_name = './output/fornecedores_2016_2017.csv'
column_types = [str, str, str, int, int, float, int, float, int, str]
import_csv_to_dynamodb(table_name, csv_file_name, colunm_names, column_types)


batch done! (row number: 1000)
batch done! (row number: 2000)
batch done! (row number: 3000)


### 7.3. Dados de Licitações
E finalmente pegamos a planilha atualizada de licitações e preparamos para envio a nossa API na AWS, para consequentemente se tornar acessível por qualquer pessoa através do nosso site.

In [ ]:
colunm_names = 'orgao retranca modalidade numero_licitacao numero_processo evento objeto data_publicacao fornecedor fornecedor_tipo fornecedor_doc data_assinatura_s validade_s validade_tipo numero_contrato valor_s data_assinatura validade valor contrato_id similar id_suspeito valor_suspeito orgao_id user_id'.split()
table_name = 'contratos'
csv_file_name = './output/licitacoes_2016_2017.csv'
column_types = [str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, str, float, float, float, str, str, int, float, str, str]
import_csv_to_dynamodb(table_name, csv_file_name, colunm_names, column_types)

batch done! (row number: 1000)
batch done! (row number: 2000)
batch done! (row number: 3000)
batch done! (row number: 4000)
batch done! (row number: 5000)
batch done! (row number: 6000)
batch done! (row number: 7000)
batch done! (row number: 8000)
